### 0. Understand your enviroment

In [ ]:
# Setup environment
using Pkg
Pkg.add("IceFloeTracker")
Pkg.add("DelimitedFiles", "Distributed")

### 1. Activate the IceFloeTracker package

First activate the project environment, load things, and set up some paths.

In [ ]:
# The notebook presently uses test images and some testing tools from the test directory.
HOME = "../.." # path to the root of the project two levels up
TEST = joinpath(HOME, "test")

# Load packages
using IceFloeTracker
using IceFloeTracker: float64, @test_approx_eq_sigma_eps, load, Gray
using DelimitedFiles # for loading test data in csv format
using Distributed # for parallel processing
addprocs(1) # add a single worker process for watershed computation
include(joinpath(TEST,"config.jl")) # loads some test data paths
include(joinpath(TEST,"test_error_rate.jl")) # loads the `test_error_rate` function

imshow(img) = Gray.(img)

## 1. Landmask generation

In [ ]:
#= 1. Load coastline
Note: #test region is the upper third of the full size image and set in the config.jl file in the test dir =#
@time coastline = load(joinpath(TEST, landmask_file))[test_region...];
# 14.208780 seconds (107.31 M allocations: 7.933 GiB, 9.32% gc time)

In [ ]:
# 2. Create landmask
@time landmask_imgs = IceFloeTracker.create_landmask(coastline);
# 78.853767 seconds (60 allocations: 98.499 MiB)

In [ ]:
# need the little imshow function to have the cell interpret the output as an image otherwise it will just print the array
imshow(landmask_imgs.non_dilated) # land is black

In [ ]:
imshow(landmask_imgs.dilated) # again, land is black

### Test landmasks

In [ ]:
# Load the ground truth landmasks
    landmask_non_dilated_expected = load(joinpath(TEST,"test_inputs/landmask_no_dilate.png"))[test_region...];
    landmask_dilated_expected = load(joinpath(TEST,"test_inputs/matlab_landmask.png"))[test_region...];

#= Compare the generated landmasks to the ground truth. No output means the test passed. The `@assert` macro throws an error if the test fails. =#
    @assert (@test_approx_eq_sigma_eps landmask_non_dilated_expected landmask_imgs.non_dilated [0,0] .001) === nothing
    @assert (@test_approx_eq_sigma_eps (landmask_dilated_expected) .!landmask_imgs.dilated [0,0] .001) === nothing

## 2. Preprocessing

Load a pair of truecolor/reflectance images

In [ ]:
# 1. Load test data

reflectance_test_image_file = "test_inputs/beaufort-chukchi-seas_falsecolor.2020162.aqua.250m.tiff"
@time reflectance_image = load(joinpath(TEST,reflectance_test_image_file))[test_region...] |> x->float64.(x) # load and convert to float64
# 5.157572 seconds (13.92 M allocations: 2.407 GiB, 3.61% gc time)

truecolor_test_image_file = "test_inputs/beaufort-chukchi-seas_truecolor.2020162.aqua.250m.tiff"
@time truecolor_image = load(joinpath(TEST,truecolor_test_image_file))[test_region...] |> x->float64.(x);
# 5.198692 seconds (19.79 M allocations: 2.756 GiB, 5.91% gc time)

In [ ]:
reflectance_image

In [ ]:
truecolor_image

Cloudmask generation

In [ ]:
# set parameters for cloudmask
prelim_threshold = Float64(110 / 255)
band_7_threshold = Float64(200 / 255)
band_2_threshold = Float64(190 / 255)
ratio_lower = 0.0
ratio_offset = 0.0
ratio_upper = 0.75

# Create cloudmask from reflectance image

@time cloudmask = IceFloeTracker.create_cloudmask(reflectance_image,
        LopezAcostaCloudMask(prelim_threshold,
                             band_7_threshold,
                             band_2_threshold,
                             ratio_lower,
                             ratio_offset,
                             ratio_upper));
# 0.272900 seconds (30 allocations: 284.867 MiB, 32.60% gc time)

imshow(cloudmask) # cloud is black

In [ ]:
# Test for cloudmask
cloudmask_expected = load(joinpath(TEST,cloudmask_test_file))[test_region...];
@assert (@test_approx_eq_sigma_eps cloudmask_expected .!cloudmask [0,0] .001) === nothing

# The `test_similarity` function provides the mismatch rate between two images (0 means they are identical) given a theshold for the difference between the two images (0.005 is the default).
test_similarity(.!cloudmask, BitMatrix(cloudmask_expected));

### 3. Create intermediate images for segmentation

In [ ]:
#= ice labels
   This output contains a vector of pixel indexes where there is obvious ice.=#
band_7_max = 5 / 255
band_2_min = 230 / 255
band_1_min = 240 / 255
band_7_max_relaxed = 10 / 255
band_1_min_relaxed = 190 / 255
possible_ice_threshold = 75 / 255

@time ice_labels = IceFloeTracker.find_ice_labels(
   reflectance_image,
   landmask_imgs.dilated;
   band_7_max,
   band_2_min,
   band_1_min,
   band_7_max_relaxed,
   band_1_min_relaxed,
   possible_ice_threshold,
)

In [ ]:
# read in matlab ice labels and test
ice_labels_expected = DelimitedFiles.readdlm(joinpath(TEST,"test_inputs/ice_labels_matlab.csv"), ',') |> vec
@assert ice_labels_expected == ice_labels

Sharpening

In [ ]:
# set parameters for imsharpen
lambda = 0.1
kappa = 75
niters = 3
nbins = 255
rblocks = 10
cblocks = 10
clip = 0.86
smoothing_param=10
intensity=2.0

# a. apply imsharpen to truecolor image using non-dilated landmask
@time sharpened_truecolor_image = IceFloeTracker.imsharpen(truecolor_image, landmask_imgs.non_dilated, lambda, kappa, niters, nbins, rblocks, cblocks, clip, smoothing_param, intensity)
# 7.579775 seconds (3.86 M allocations: 3.822 GiB, 9.46% gc time, 34.64% compilation time)

# b. apply imsharpen to sharpened truecolor img using dilated landmask
@time sharpened_gray_truecolor_image = IceFloeTracker.imsharpen_gray(sharpened_truecolor_image, landmask_imgs.dilated)
# 0.122422 seconds (389.67 k allocations: 202.584 MiB, 68.19% compilation time)

# Test sharpened_gray_truecolor_image against matlab_sharpened_gray
matlab_sharpened_gray_file = "test_inputs/matlab_sharpened.png"
matlab_sharpened_gray = float64.(load(joinpath(TEST, matlab_sharpened_gray_file)))
@assert (@test_approx_eq_sigma_eps matlab_sharpened_gray sharpened_gray_truecolor_image [0, 0] 0.046) === nothing

In [ ]:
imshow(sharpened_truecolor_image)

In [ ]:
imshow(sharpened_gray_truecolor_image)

Normalization

In [ ]:
# Normalization
@time normalized_image = IceFloeTracker.normalize_image(
    sharpened_truecolor_image, sharpened_gray_truecolor_image, landmask_imgs.dilated)
# Expected runtime: 46.354389 seconds (1.20 G allocations: 29.722 GiB, 7.13% gc time, 0.32% compilation time)

In [ ]:
# Test normalized image
matlab_normalized_img_file = "test_inputs/matlab_normalized.png"
norm_image_expected = float64.(load(joinpath(TEST,matlab_normalized_img_file))[test_region...])
@assert (@test_approx_eq_sigma_eps normalized_image norm_image_expected [0, 0] 0.045) === nothing

Ice water discrimination

In [ ]:
#= Icewater discrimination
    Note that this function mutates the landmask.dilated. =#

# set parameters
floes_threshold = 100 / 255
mask_clouds_lower = 17 / 255
mask_clouds_upper = 30 / 255
kurt_thresh_lower = 2
kurt_thresh_upper = 8
skew_thresh = 4
st_dev_thresh_lower = 84 / 255
st_dev_thresh_upper = 98.9 / 255
clouds_ratio_threshold = 0.02
differ_threshold = 0.6
nbins = 155

@time ice_water_discrim = IceFloeTracker.discriminate_ice_water(
    reflectance_image, normalized_image, copy(landmask_imgs.dilated), # pass a copy of landmask
    cloudmask, floes_threshold, mask_clouds_lower, mask_clouds_upper, kurt_thresh_lower, kurt_thresh_upper, skew_thresh, st_dev_thresh_lower, st_dev_thresh_upper, clouds_ratio_threshold, differ_threshold, nbins)

# 3.059782 seconds (7.03 M allocations: 1.376 GiB, 30.85% gc time, 65.98% compilation time)

In [ ]:
# Test ice_water_discrim.ice_water
matlab_ice_water_discrim_file = "test_inputs/matlab_ice_water_discrim.png"
ice_water_discrim_expected = float64.(load(joinpath(TEST,matlab_ice_water_discrim_file))[test_region...])
@assert (@test_approx_eq_sigma_eps ice_water_discrim ice_water_discrim_expected [0, 0] 0.065) === nothing

## 4. Segmentation

In [ ]:
# segmentation_A
@time segA = IceFloeTracker.segmentation_A(IceFloeTracker.segmented_ice_cloudmasking(
    ice_water_discrim, cloudmask, ice_labels
))
# 83.245360 seconds (963.75 M allocations: 69.364 GiB, 10.71% gc time, 0.59% compilation time: 22% of which was recompilation)

In [ ]:
imshow(segA)

### Note
Here is the first significant source of error introduced due to the implementation of the kmeans algorithm used for the package. For details see `src/segmentation_a_direct.jl` and/or https://juliastats.org/Clustering.jl/dev/kmeans.html#Clustering.kmeans.

In [ ]:
# Test segmentation_A
segA_expected = convert(BitMatrix,float64.(load(joinpath(TEST,"test_inputs/matlab_segmented_A.png"))))
@assert test_similarity(segA_expected, segA, 0.1005)

In [ ]:
@time segB = IceFloeTracker.segmentation_B(sharpened_gray_truecolor_image, cloudmask, segA)
# 1.264987 seconds (24.14 M allocations: 1.041 GiB, 0.36% compilation time)

In [ ]:
# Test segmentation_B
matlab_ice_intersect = convert(
        BitMatrix, load(joinpath(TEST,"test_inputs/matlab_segmented_c.png")
    ))

    matlab_not_ice_mask = float64.(load(joinpath(TEST,"test_inputs/matlab_I.png")))

@assert (@test_approx_eq_sigma_eps segB.not_ice matlab_not_ice_mask [0, 0] 0.055) === nothing
@assert test_similarity((matlab_not_ice_mask .> 0.499), segB.not_ice_bit, 0.05)
@assert test_similarity(matlab_ice_intersect, segB.ice_intersect, 0.08)

In [ ]:
imshow(segB.not_ice)

In [ ]:
imshow(segB.not_ice_bit)

In [ ]:
imshow(segB.ice_intersect)

In [ ]:
# Generate watersheds
@everywhere using IceFloeTracker: watershed_ice_floes
@time watersheds_segB = pmap(IceFloeTracker.watershed_ice_floes, [segB.not_ice_bit, segB.ice_intersect])
# 423.326505 seconds (6.43 M allocations: 327.860 GiB, 8.08% gc time, 0.53% compilation time)

In [ ]:
watershed_intersect = IceFloeTracker.watershed_product(watersheds_segB...)

In [ ]:
# Test watershed_intersect
matlab_watershed_D = convert(BitMatrix, load(joinpath(TEST,"test_inputs/matlab_watershed_D.png")))
matlab_watershed_E = convert(BitMatrix, load(joinpath(TEST,"test_inputs/matlab_watershed_E.png")))
matlab_watershed_intersect = convert(BitMatrix, load(joinpath(TEST,"test_inputs/matlab_watershed_intersect.png")))

## Tests with Matlab inputs
@assert test_similarity(matlab_watershed_D, watersheds_segB[1], 0.15)
@assert test_similarity(matlab_watershed_E, watersheds_segB[2], 0.15)
@assert test_similarity(matlab_watershed_intersect, watershed_intersect , 0.033)

In [ ]:
@time isolated_floes = IceFloeTracker.segmentation_F(
    segB.not_ice,
    segB.ice_intersect,
    watershed_intersect,
    ice_labels,
    cloudmask,
    landmask_imgs.dilated,
)
# 445.557423 seconds (2.81 G allocations: 89.153 GiB, 2.79% gc time, 0.00% compilation time)

In [ ]:
# Test isolated_floes
matlab_BW7 = load(joinpath(TEST,"test_inputs/matlab_BW7.png")) .> 0.499

@assert test_similarity(matlab_BW7, isolated_floes, 0.143)

@assert test_similarity(matlab_BW7[ice_floe_test_region...], isolated_floes[ice_floe_test_region...], 0.0705)


In [ ]:
imshow(isolated_floes)

In [ ]:
imshow(matlab_BW7)

In [ ]:
imshow(isolated_floes[ice_floe_test_region...])

In [ ]:
imshow(matlab_BW7[ice_floe_test_region...])